# standard transforms

> Fill in a module description here

In [ ]:
#| default_exp tfsm/standard

In [ ]:
#| export 
import torch 
import fastcore.all as fc
from typing import Union, List
from voxdet.tfsm.utils import chwd_2_corner

In [ ]:
import numpy as np

In [ ]:
#| export 
class BaseT(torch.nn.Module):
    def __init__(self): super().__init__()
    def forward(self, img:Union[List[dict], dict]):
        if isinstance(img, list): return [self.apply(i) for i in img] 
        else: return self.apply(img)
    def apply(self, img:dict): raise NotImplementedError

In [ ]:
#| export 
class StandardT(BaseT):
    def __init__(self, src_mode:str, img_src_mode: str): 
        # converts bbox to zyxzyx format and img_src_mode to zyx format
        fc.store_attr() 
        super().__init__()
        self._check_bbox_mode()
        self._check_img_mode()
    
    __repr__ = fc.basic_repr(flds="src_mode, img_src_mode")

    def _check_bbox_mode(self):
        if self.src_mode not in ["yxzhwd"]: raise NotImplementedError(f"src_mode {self.src_mode} not implemented")
        
    def _check_img_mode(self):
        if self.img_src_mode not in ["zyx", "xyz"]: raise NotImplementedError(f"img_src_mode {self.img_src_mode} not implemented")
        
    def forward(self, img:Union[List[dict], dict]):
        if isinstance(img, list): return [self.apply(i) for i in img] 
        else: return self.apply(img)
    
    def apply(self, img: dict):
        assert "images" in img.keys(), "images are not present."
        nimg = {}
        nimg["images"] = self.apply_img(img["images"].copy())
        if "boxes" in img.keys(): nimg["boxes"] = self.apply_bbox(img["boxes"].copy())
        for i in img.keys(): 
            if i not in nimg.keys(): nimg[i] = img[i]
        return nimg
    
    def apply_img(self, img):
        out = img.copy()
        if self.img_src_mode == "zyx": return out 
        if self.img_src_mode == "xyz": return out.transpose((2, 1, 0))
        raise NotImplementedError(f"img_src_mode: {self.img_src_mode}, img_dst_mode: {self.img_dst_mode} not implemented")
    
    
    def apply_bbox(self, bbox):
        #dest mode is always xyzxyz
        if self.src_mode == "yxzhwd": return chwd_2_corner(bbox)[:, [2, 0, 1, 5, 3, 4]]
        raise NotImplementedError(f"src_mode: {self.src_mode}, dst_mode: {self.dst_mode} not implemented")
    
    #TODO: we should Ideally reverse this to original dimensions. 
    def reverse_apply(self, img:dict): return img 

In [ ]:
fc.test_fail(StandardT, args=("xyzxyz", "xyz"))

In [ ]:
tfm = StandardT(src_mode="yxzhwd", img_src_mode="zyx")
tfm

In [ ]:
x = {}
x["images"] = np.zeros((32, 42, 52))
x["boxes"] = np.asarray([[10, 20, 30, 10, 20, 30]])
y = tfm(x)
assert fc.all_equal(y["images"].shape, x["images"].shape), "shape changes with img_src and img_dst modes are same"
assert fc.all_equal(y["boxes"], np.asarray([[15, 5, 10, 45, 15, 30]])), "bbox transforms are wrong"

In [ ]:
x = {}
x["images"] = np.zeros((292, 370, 370))
x["boxes"] = np.asarray([[203.06635 ,  96.83591 ,  58.75    ,  10.83984 ,  13.730464, 10.]])
y = tfm(x) #[yxzhwd] [2, 0, 1, 5, 3, 4]

In [ ]:
chwd_2_corner(x["boxes"])

In [ ]:
y["boxes"]

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()